#### Challenge

In [ ]:
import numpy as np
from random import SystemRandom
from Crypto.Util.number import bytes_to_long, long_to_bytes

# dimension
n = 64
# number of samples in public key
m = 512
dtype = np.uint16

random = SystemRandom()
sigma = 2.3
def normal(): return round(random.gauss(0, sigma))
def binary(): return random.randrange(2)

FLAG = b"crypto{?????????????????????????????????????????}"

def uniform(shape):
    buffer = [random.randrange(0, 1 << 16) for i in range(np.prod(shape))]
    return np.array(buffer, dtype=dtype).reshape(shape)


def sample(shape, dist):
    return np.fromfunction(np.vectorize(lambda *_: dist()), shape).astype(dtype)


def keygen():
    A = uniform((n, m))
    s = uniform((n,))
    pk = np.vstack((A, s @ A + 2*sample((m,), normal)))
    sk = np.append(-s, 1).astype(dtype)
    return pk, sk


def encrypt(pk, msg):
    c = pk @ sample((m,), binary) + np.append(np.zeros(n), msg)
    return c.astype(dtype)


def decrypt(sk, c):
    return sk.dot(c) & 1


pk, sk = keygen()

msg = np.fromiter([int(i) for i in "{:0{}b}".format(
    bytes_to_long(FLAG), 8 * len(FLAG))], dtype)
ciphertexts = np.vstack([encrypt(pk, b) for b in msg])

np.savetxt("ciphertexts.txt", ciphertexts, fmt="%d")
np.savetxt("public_key.txt", pk, fmt="%d")


In [ ]:
from requests import Session
import numpy as np

def decrypt(sk, c):
    return sk.dot(c) & 1

def readlines(URL):
    return Session().get(URL).text.strip().split("\n")

public_key_URL = "https://cryptohack.org/static/challenges/public_key_dbd7dcf356ea5c595d390470faad2c29.txt"
ciphertexts_URL = "https://cryptohack.org/static/challenges/ciphertexts_453500a870e03399fe8e5ed3674b6030.txt"

pk = [[int(x) for x in line.split()] for line in readlines(public_key_URL)]
ct = [np.array([int(x) for x in line.split()]) for line in readlines(ciphertexts_URL)]
sk = np.array([int(i) for i in Matrix(GF(2), pk).left_kernel()[1]])
msg = [str(decrypt(sk, c)) for c in ct]
print(bytes.fromhex(hex(int("".join(msg), 2))[2:]).decode())



# This script is a lattice-based encryption system and solves for the flag using the public key and ciphertexts. It begins by generating a public key (`pk`) and a secret key (`sk`) using random samples. The public key is created with a matrix `A` and a random vector `s`, and the secret key is constructed by combining the negation of `s` with `1`. The encryption function takes the public key and a message, computes the ciphertext by performing matrix multiplication with a random binary vector and appending the message. Decryption is performed by calculating the dot product of the secret key and the ciphertext, followed by modulo 2 arithmetic to recover the message. The script then encrypts a binary representation of the `FLAG` and stores the ciphertexts along with the public key. In the second part of the script, it fetches the public key and ciphertexts from URLs using the `requests` library, reconstructs the secret key by computing the left kernel of the public key matrix, and decrypts the ciphertexts to retrieve the flag.

crypto{flavortext-flag-coprime-regev-yadda-yadda}
